<a href="https://colab.research.google.com/github/Unessam/Identify-the-number/blob/master/Identify_the_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [13]:
from google.colab import files
csv_files=files.upload()
for csv in csv_files.keys():
  print('file name: {}, file shape: {}'.format(csv, len(csv_files[csv])))

Saving sample_submission.csv to sample_submission (1).csv
Saving test.csv to test (1).csv
Saving train.csv to train (1).csv
file name: sample_submission.csv, file shape: 252015
file name: test.csv, file shape: 210009
file name: train.csv, file shape: 576905


In [15]:
train_im=files.upload()

Saving im_train.zip to im_train.zip


In [ ]:
!unzip im_train.zip

In [17]:
train=pd.read_csv('train.csv')
train_org=train.copy()
test=pd.read_csv('test.csv')
test_org=test.copy()
submission=pd.read_csv('sample_submission.csv')

In [28]:
train_image = []
for im_name in tqdm(train['filename']):
    img = image.load_img('im_train/'+im_name, target_size=(28,28,1), color_mode="grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 49000/49000 [00:13<00:00, 3571.49it/s]


In [20]:
y=train['label'].values
y = to_categorical(y)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [32]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [33]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1225/1225 [==============================] - 107s 87ms/step - loss: 0.2534 - accuracy: 0.9228 - val_loss: 0.0690 - val_accuracy: 0.9787
Epoch 2/10
1225/1225 [==============================] - 107s 87ms/step - loss: 0.1026 - accuracy: 0.9691 - val_loss: 0.0552 - val_accuracy: 0.9843
Epoch 3/10
1225/1225 [==============================] - 106s 87ms/step - loss: 0.0777 - accuracy: 0.9769 - val_loss: 0.0489 - val_accuracy: 0.9862
Epoch 4/10
1225/1225 [==============================] - 107s 87ms/step - loss: 0.0632 - accuracy: 0.9805 - val_loss: 0.0436 - val_accuracy: 0.9866
Epoch 5/10
1225/1225 [==============================] - 107s 87ms/step - loss: 0.0538 - accuracy: 0.9835 - val_loss: 0.0448 - val_accuracy: 0.9870
Epoch 6/10
1225/1225 [==============================] - 107s 87ms/step - loss: 0.0489 - accuracy: 0.9852 - val_loss: 0.0443 - val_accuracy: 0.9873
Epoch 7/10
1225/1225 [==============================] - 107s 87ms/step - loss: 0.0390 - accuracy: 0.9875 - val_loss: 0

In [35]:
test_im=files.upload()

Saving im_test.zip to im_test.zip


In [ ]:
!unzip im_test.zip

In [38]:
test_image=[]
for im_name in tqdm(test['filename']):
  img=image.load_img('im_test/'+im_name, target_size=(28, 28, 1), color_mode='grayscale')
  img=image.img_to_array(img)
  img=img/255
  test_image.append(img)
Test=np.array(test_image)

100%|██████████| 21000/21000 [00:05<00:00, 3907.27it/s]


In [40]:
prediction = model.predict_classes(Test)

In [41]:
submission['label'] = prediction
submission.to_csv('First_submission.csv', header=True, index=False)